# Voice Assistant

### Import Libraries:

In [1]:
import speech_recognition as sr #convert speech to text
import datetime #for fetching date and time
import playsound # to play saved mp3 file 
from gtts import gTTS # google text to speech
import json
import time
import noisereduce as nr
import cv2
import pyaudio
import numpy as np
import os # to save/open files 

### Respond Your Voice:

In [2]:
def talk():
    aud=sr.Recognizer()
    with sr.Microphone() as source:
        respond("speak now")
        audio=aud.listen(source)
        data=""
        try:
            data=aud.recognize_google(audio)
            print("Your question is, " + data)
            
        except sr.UnknownValueError:
            respond("Sorry, I didn't catch that. Could you please repeat?")
            
        except sr.RequestError:
            respond("Sorry, my speech service is not available at the moment.")
            
    return data

In [3]:
def respond(output):
    num=0
    print(output)
    num += 1
    response=gTTS(text=output, lang='en')
    file = str(num)+".mp3"
    response.save(file)
    playsound.playsound(file, True)
    os.remove(file)

### Wake Word:

In [4]:
def wakeWord(text):
    WAKE_WORDS = ['cm'] 
    text = text.lower() 
    
    for phrase in WAKE_WORDS:
        if phrase in text:
            return True
        
    return False

### Remove Background Noise:

In [5]:
def remove_background_noise(duration=5):

    chunk = 1024
    format = pyaudio.paInt16
    channels = 1
    sample_rate = 16000

    p = pyaudio.PyAudio()
    stream = p.open(format=format,
                    channels=channels,
                    rate=sample_rate,
                    input=True,
                    frames_per_buffer=chunk)

    frames = []
    for i in range(int(sample_rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)

    audio_array = np.frombuffer(b''.join(frames), dtype=np.int16)

    reduced_noise = nr.reduce_noise(y=audio_array, sr=sample_rate)

    stream.stop_stream()
    stream.close()
    p.terminate()

    return reduced_noise

### Face Detection:

In [6]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [7]:
def detect_faces():

    video_capture = cv2.VideoCapture(0)

    while True:
    
        ret, frame = video_capture.read()

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        if len(faces) > 0:
            respond("Face Detected")
            wishme()
            break
            
        if len(faces) == 0 :
            respond("Face Not Detected")
            respond("Please come infront of me and say a wakeword")
            video_capture.release()
            cv2.destroyAllWindows()
            return
        
        for (x, y, w, h) in faces:
            
            if w > 100 and h > 100:  
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            elif w > 50 and h > 50: 
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            else:
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)


        cv2.imshow('Face Detection', frame)
        
        if cv2.waitKey(3000) != -1:
            break

    video_capture.release()
    
    cv2.destroyAllWindows()

### Welcome Speech :

In [8]:
def wishme():
    
    hour = datetime.datetime.now().hour
    if hour >= 4 and hour < 12:
        respond("Good Morning")
    elif hour >= 12 and hour < 16:
        respond("Good Afternoon")
    elif hour >= 16 and hour < 24:
        respond("Good Evening")
    else:
        respond("Good Night, See You Tommorrow")
        
    respond("Welcome to svm jewellery shop")
    
    respond("Hello, I am your svm Jewellery Shop Voice Assistant, How can I help you?")

### Get Date and Time:

In [9]:
def get_date():
    now = datetime.datetime.now()
    date = now.strftime("%A, %B %d, %Y")
    respond("Today's Date is " + date)

In [10]:
def get_time():
    now = datetime.datetime.now()
    time = now.strftime("%I:%M %p")
    respond("The Current Time is " + time)

### Import Data from JSON File:

In [11]:
jewellery_data = {
    
    "collections": {
        
        "gold":  { 
            
            "gold rate" : [
                    {"gold_per_gram" : "4800 rupees + GST(Applicable)", "gold_eight_gram" : "45000 rupees + GST(Applicable)"}
            ],
            
            "items": [
                {"name": "necklace"},
                {"name": "bracelet"},
                {"name": "ring"}
            ],
            
            "offers": ["10% offer on all items"
            ]
        },
        
        "silver": {
            
            "silver rate" : [
                    {"silver_per_gram" : "77 rupees + GST(Applicable)", "silver_eight_gram" : "675 rupees + GST(Applicable)"}
                 ],
            
            "items": [
                {"name": "earrings"},
                {"name": "pendant"},
                {"name": "bangle"}
            ],
            
            "offers": ["15% offer on all items"
            ]
        },
        
        "diamond": {
            
            "diamond rate" : [
                    {"diamond_per_carat" : "6500 rupees + GST(Applicable)", "diamond_per_gram" : "300000 rupees + GST(Applicable)"}
                 ],
            
            "items": [
                {"name": "earrings"},
                {"name": "ring"},
                {"name": "pendant"}
            ],
            
            "offers": ["5% offer on all items"
            ]
        },
        
        "platinum" : {
            
            "platinum rate" : [
                    {"platinum_per_gram" : "2454 rupees + GST(Applicable)", "platinum_eight_gram" : "23465 rupees + GST(Applicable)"}
                 ],
            
            "items": [
                {"name": "chain"},
                {"name": "bracelet"},
                {"name": "ring"}
            ],
            
            "offers" : ["No offers available for this collection"],
        }       
    }
}

### Read JSON File :

In [12]:
file_path = "jewellery_data.json"

with open(file_path, "w") as file:
    json.dump(jewellery_data, file, indent=4)

In [13]:
def load_jewellery_data():
    with open("jewellery_data.json") as file:
        jewellery_data = json.load(file)
    return jewellery_data

jewellery_data = load_jewellery_data()

### Show all Collection:

In [14]:
def find_collection():
    collections = jewellery_data["collections"]
    respond("We have the following collections:")
    for collection in collections:
        
        respond(collection)

### Show Collection Price:

In [15]:
def gold_rate():
    collection = "gold"
    if collection in jewellery_data["collections"]:
        collection_data = jewellery_data["collections"][collection]
        if "gold rate" in collection_data:
            rate = collection_data["gold rate"]
            respond(f"The {collection} collection has the following rate:")
            for item in collection_data["gold rate"]:
                respond(f" one gram rate is {item['gold_per_gram']} :  eight gram rate is {item['gold_eight_gram']}")

In [16]:
def silver_rate():
    collection = "silver"
    if collection in jewellery_data["collections"]:
        collection_data = jewellery_data["collections"][collection]
        if "silver rate" in collection_data:
            rate = collection_data["silver rate"]
            respond(f"The {collection} collection has the following rate:")
            for item in collection_data["silver rate"]:
                respond(f" one gram rate is {item['silver_per_gram']} :  eight gram rate is  {item['silver_eight_gram']}")

In [17]:
def diamond_rate():
    collection = "diamond"
    if collection in jewellery_data["collections"]:
        collection_data = jewellery_data["collections"][collection]
        if "diamond rate" in collection_data:
            rate = collection_data["diamond rate"]
            respond(f"The {collection} collection has the following rate:")
            for item in collection_data["diamond rate"]:
                respond(f" one carat rate is {item['diamond_per_carat']} :  eight gram rate is  {item['diamond_per_gram']}")

In [18]:
def platinum_rate():
    collection = "platinum"
    if collection in jewellery_data["collections"]:
        collection_data = jewellery_data["collections"][collection]
        if "platinum rate" in collection_data:
            rate = collection_data["platinum rate"]
            respond(f"The {collection} collection has the following rate:")
            for item in collection_data["platinum rate"]:
                respond(f"one gram rate is {item['platinum_per_gram']} : eight gram rate is  {item['platinum_eight_gram']}")

### Show Collection Items:

In [19]:
def gold_items():
    collection = "gold"
    if collection in jewellery_data["collections"]:
        collection_data = jewellery_data["collections"][collection]
        if "items" in collection_data:
            items = collection_data["items"]
            respond(f"The {collection} collection has the following items:")
            for item in items:
                respond(item["name"])
                
            respond("The items are available in the ground floor")

In [20]:
def silver_items():
    collection = "silver"
    if collection in jewellery_data["collections"]:
        collection_data = jewellery_data["collections"][collection]
        if "items" in collection_data:
            items = collection_data["items"]
            respond(f"The {collection} collection has the following items:")
            for item in items:
                respond(item["name"])
                
            respond("The items are available in the first floor")

In [21]:
def diamond_items():
    collection = "diamond"
    if collection in jewellery_data["collections"]:
        collection_data = jewellery_data["collections"][collection]
        if "items" in collection_data:
            items = collection_data["items"]
            respond(f"The {collection} collection has the following items:")
            for item in items:
                respond(item["name"])
                
            respond("The items are available in the second floor")

In [22]:
def platinum_items():
    collection = "platinum"
    if collection in jewellery_data["collections"]:
        collection_data = jewellery_data["collections"][collection]
        if "items" in collection_data:
            items = collection_data["items"]
            respond(f"The {collection} collection has the following items:")
            for item in items:
                respond(item["name"])
                
            respond("The items are available in the second floor")

### Show All Offers:

In [23]:
def offers(collection_name):
    if collection_name not in jewellery_data["collections"]:
        return "Collection not found."

    offers = jewellery_data["collections"][collection_name]["offers"]

    offer_list = "\n".join(offers)
    return f"The offers in the {collection_name} collection are:\n{offer_list}"

In [24]:
def show_all_offers():
    respond(offers("gold"))
    respond(offers("silver"))
    respond(offers("diamond"))
    respond(offers("platinum"))

### Main Program :

In [29]:
if __name__=='__main__':
    
    last_command_time = time.time()
        
    while(1):

        video_capture = cv2.VideoCapture(0)
        ret, frame = video_capture.read()

        text=talk().lower()
        current_time = time.time()
        
        remove_background_noise(duration=5)
        
        if text==0:
            continue
            
        if text == "":  
            if current_time - last_command_time > 10:
                respond("No commands received. Exiting...")
                break
            
        if (wakeWord(text)==True):
            detect_faces()
            
        elif "stop" in text or "exit" in text or "bye" in text or "thanks" in text or "thank you" in text or "okey bye" in text:
            respond("Thank you for your pleasant time...")
            break       
            
        elif "who are you" in text:
            respond("I'm your svm jewellery shop voice assistant")

        elif "how are you" in text:
            respond("I'm fine sir, what about you")
            
        elif "date" in text or "what is current date" in text or "today date" in text:
            get_date()
            
        elif  "time" in text or "what is current time" in text or "today time" in text:
            get_time()
            
        elif  "what is current date and time" in text or "date and time" in text or "today date and time" in text:
            get_date()
            get_time()
            
        elif "show me collection" in text or "list of collection" in text or "collections" in text:
            find_collection()
            
        elif "gold rate" in text or "what is today's gold rate" in text or "current gold rate" in text or "gold price" in text or "what is today's gold price" in text or "current gold price" in text:
            gold_rate()
            
        elif "silver rate" in text or "what is today's silver rate" in text or "current silver rate" in text or "silver price" in text or "what is today's silver price" in text or "current silver price" in text:
            silver_rate()
            
        elif "diamond rate" in text or "what is today's diamond rate" in text or "current diamond rate" in text or "diamond price" in text or "what is today's diamond price" in text or "current diamond price" in text:
            diamond_rate()
            
        elif "platinum rate" in text or "what is today's platinum rate" in text or "current platinum rate" in text or "platinum price" in text or "what is today's platinum price" in text or "current platinum price" in text:
            platinum_rate()
            
        elif "gold items" in text or "gold products" in text or "show gold items" in text:
            gold_items()
            
        elif "silver items" in text or "silver products" in text or "show silver items" in text:
            silver_items()
            
        elif "diamond items" in text or "diamond products" in text or "show diamond items" in text:
            diamond_items()
            
        elif "platinum items" in text or "platinum products" in text or "show platinum items" in text:
            platinum_items()
            
        elif "offers" in text or "is there any offer avaliable" in text or "today offers" in text:
            show_all_offers()
            
        else:
            respond("Sorry, I didn't understand that. Please repeat the question.")

speak now
Your question is, CM
Face Detected
Good Evening
Welcome to svm jewellery shop
Hello, I am your svm Jewellery Shop Voice Assistant, How can I help you?
speak now
Your question is, how are you
I'm fine sir, what about you
speak now
Your question is, are you
Sorry, I didn't understand that. Please try again.
speak now
Your question is, date and time
Today's Date is Friday, June 02, 2023
speak now
Your question is, movie collection
Sorry, I didn't understand that. Please try again.
speak now
Your question is, date and time
Today's Date is Friday, June 02, 2023
speak now
Your question is, correction
Sorry, I didn't understand that. Please try again.
speak now
Your question is, collection
Sorry, I didn't understand that. Please try again.
speak now
Your question is, collections
We have the following collections:
gold
silver
diamond
platinum
speak now
Your question is, gold rate
The gold collection has the following rate:
 one gram rate is 4800 rupees + GST(Applicable) :  eight gram